# Software Engineering Assessment

**Problem Statement**

Currently have a Power Automate flow that takes data from multiple sources and merges them into one output to be used by finance teams. Flow is slow to run and prone to failure, a Python ETL process to be spun up to replace it.

## Setup Instructions

This code assumes you have only loaded Pipeline.ipynb into Google colab, if you have already uploaded the below files you can skip the next step and move straight to Installign requirements.txt

Run the following code block and upload the following files from the Zip Archive:

* PayRates.csv
* Staff.csv
* Teams.csv
* requirements.txt

In [ ]:
from google.colab import files
uploaded = files.upload()

The following code installs the following required libraries:

*   Pandas
*   Faker
*   YData-profiling

In [ ]:
!pip install -r requirements.txt

In [ ]:
# Importing installed libraries
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport

# As unittest is built into python it does not need to be installed like previous libraries
import unittest

## Import Data

This step reads the three source data files into Pandas dataframes, these files contain synthetic data based on production outputs used for the pipeline used internally within my company.

In [ ]:
df_staff = pd.read_csv('Staff.csv')
df_teams = pd.read_csv('Teams.csv')
df_payrates = pd.read_csv('PayRates.csv')

# For troubleshooting
print(df_staff.head())
print(df_teams.head())
print(df_payrates.head())

## YData Auto EDA

This step generates and downloads autoEDA reports for each of the three dataframes

In [ ]:
# Creating the Profile Reports
staff_profile = ProfileReport(df_staff, title="Staff Profiling Report")
teams_profile = ProfileReport(df_staff, title="Teams Profiling Report")
pay_profile = ProfileReport(df_staff, title="Pay Profiling Report")

# Saving Profile Reports as html files in the colab space
staff_profile.to_file("staff_report.html")
teams_profile.to_file("teams_report.html")
pay_profile.to_file("pay_report.html")

# Downloading the html reports
files.download("staff_report.html")
files.download("teams_report.html")
files.download("pay_report.html")

##Data Prep

Initial Data Cleansing

In [ ]:
# Dropping duplicate IDs
df_staff = df_staff[['ID','Full Name', 'Location']].drop_duplicates(subset=['ID'])

#Sorting the table on ID to make the output easier to read
df_staff.sort_values(by='ID', inplace=True)

# reseting the index to ensure cleaning joining in future steps
df_staff.reset_index(drop=True, inplace=True)

# For Troubleshooting
print(df_staff.head())

In [ ]:
# Repeating above step for teams table
df_teams = df_teams[['ID','Department', 'Team']].drop_duplicates(subset=['ID'])
df_teams.sort_values(by='ID', inplace=True)
df_teams.reset_index(drop=True, inplace=True)
print(df_teams.head())

In [ ]:
# Repeating above step for payrates table
df_payrates = df_payrates[['ID','Pay Rate']].drop_duplicates(subset=['ID'])
df_payrates.sort_values(by='ID', inplace=True)
df_payrates.reset_index(drop=True, inplace=True)
print(df_payrates.head())

## Data Merging, Final Cleanup And Export


In [ ]:
# Merging in team data, left joining on ID
df_merged = pd.merge(df_staff, df_teams, on='ID', how='left')

# Merging in payrate data, left joining on ID
df_merged = pd.merge(df_merged, df_payrates, on='ID', how='left')

# Output for troubleshooting
print(df_merged.head())
print(df_merged.info())

### Functions

In [ ]:
# Adding leading zeros to ID Column, ensuring it's always 8 digits long

"""
Args:
    df: pandas DataFrame.
    column_name: Name of the column to add zeros to.
    total_digits: The total number of digits for that column.

  Returns:
    DataFrame with the column updated to have leading zeros.
"""

def add_leading_zeros(df, column_name, total_digits=8):

  df[column_name] = df[column_name].astype(str).str.zfill(total_digits)
  return df

df_merged = add_leading_zeros(df_merged, 'ID')
print(df_merged['ID'].head())

In [ ]:
# This will remove common titles like Mr/Mrs/Dr from the Full Name column, this is something that could be resolved during generation of the synthetic data however
# I have left the titles in the data to provide a good example of a function that can resolve this type of source data issue.

"""
  Args:
    df: pandas DataFrame.
    column_name: Name of the column to remove titles from.

  Returns:
    DataFrame with the column updated without titles.
"""

def remove_titles(df, column_name):

# Ensures the column has data, is in the dataframe and is formated as a string
  if df.empty or column_name not in df.columns or not pd.api.types.is_string_dtype(df[column_name]):
      return df
# List of common titles, these appear to be the only titles included in the source data
  titles = ['Mr', 'Mrs', 'Ms', 'Dr', 'mr', 'mrs', 'ms', 'dr']

# Removes the titles and any following periods from the data
  df[column_name] = df[column_name].str.replace(r'\b(' + '|'.join(titles) + r')\b\.?', '', regex=True).str.strip()
  return df

df_merged = remove_titles(df_merged, 'Full Name')
print(df_merged['Full Name'].head())

### Final Output, in the production pipeline this outputs directly into a SharePoint folder for the purposes of this assessment however it outputs and downloads a CSV

In [ ]:
df_merged.to_csv('final_output.csv', index=False)
files.download('final_output.csv')

## Unit Testing

### Add Leading Zeros Testing

In [ ]:
class TestAddLeadingZeros(unittest.TestCase):

    def test_add_zeros_to_short_id(self):
        df = pd.DataFrame({'ID': [123, 4567, 89]})
        df_modified = add_leading_zeros(df, 'ID', total_digits=8)
        expected_ids = ['00000123', '00004567', '00000089']
        self.assertEqual(list(df_modified['ID']), expected_ids)

    def test_id_already_correct_length(self):
        df = pd.DataFrame({'ID': [12345678, 98765432]})
        df_modified = add_leading_zeros(df, 'ID', total_digits=8)
        expected_ids = ['12345678', '98765432']
        self.assertEqual(list(df_modified['ID']), expected_ids)

    def test_id_longer_than_total_digits(self):
        df = pd.DataFrame({'ID': [123456789, 9876543210]})
        df_modified = add_leading_zeros(df, 'ID', total_digits=8)
        # zfill does not truncate, it only adds zeros
        expected_ids = ['123456789', '9876543210']
        self.assertEqual(list(df_modified['ID']), expected_ids)

    def test_empty_dataframe(self):
        df = pd.DataFrame({'ID': []})
        df_modified = add_leading_zeros(df, 'ID', total_digits=8)
        expected_ids = []
        self.assertEqual(list(df_modified['ID']), expected_ids)

    def test_column_with_strings(self):
        df = pd.DataFrame({'ID': ['abc', 'defg']})
        df_modified = add_leading_zeros(df, 'ID', total_digits=8)
        expected_ids = ['00000abc', '0000defg']
        self.assertEqual(list(df_modified['ID']), expected_ids)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

### Remove Titles Testing

In [ ]:
class TestRemoveTitles(unittest.TestCase):

    def test_remove_mr(self):
        df = pd.DataFrame({'Full Name': ['Mr. John Smith','mr. John Smith','Mr John Smith','mr John Smith', 'John Smith']})
        df_modified = remove_titles(df, 'Full Name')
        expected_names = ['John Smith', 'John Smith', 'John Smith', 'John Smith', 'John Smith']
        self.assertEqual(list(df_modified['Full Name']), expected_names)

    def test_remove_mrs(self):
        df = pd.DataFrame({'Full Name': ['Mrs. Jane Doe','Mrs Jane Doe','mrs. Jane Doe','mrs Jane Doe', 'Jane Doe']})
        df_modified = remove_titles(df, 'Full Name')
        expected_names = ['Jane Doe', 'Jane Doe', 'Jane Doe', 'Jane Doe', 'Jane Doe']
        self.assertEqual(list(df_modified['Full Name']), expected_names)

    def test_remove_ms(self):
        df = pd.DataFrame({'Full Name': ['Ms. Fonda Lee', 'Ms Fonda Lee','ms. Fonda Lee','ms Fonda Lee','Fonda Lee']})
        df_modified = remove_titles(df, 'Full Name')
        expected_names = ['Fonda Lee','Fonda Lee','Fonda Lee','Fonda Lee','Fonda Lee']
        self.assertEqual(list(df_modified['Full Name']), expected_names)

    def test_remove_dr(self):
        df = pd.DataFrame({'Full Name': ['Dr. Gregory House','Dr Gregory House','dr. Gregory House','dr Gregory House','Gregory House']})
        df_modified = remove_titles(df, 'Full Name')
        expected_names = ['Gregory House','Gregory House','Gregory House','Gregory House','Gregory House']
        self.assertEqual(list(df_modified['Full Name']), expected_names)

    def test_multiple_titles_not_present(self):
        df = pd.DataFrame({'Full Name': ['John Smith', 'Jane Doe']})
        df_modified = remove_titles(df, 'Full Name')
        expected_names = ['John Smith', 'Jane Doe']
        self.assertEqual(list(df_modified['Full Name']), expected_names)

    def test_empty_dataframe(self):
        df = pd.DataFrame({'Full Name': []})
        df_modified = remove_titles(df, 'Full Name')
        expected_names = []
        self.assertEqual(list(df_modified['Full Name']), expected_names)

    def test_names_with_dots_not_titles(self):
        df = pd.DataFrame({'Full Name': ['J.R.R. Tolkien', 'G.R.R. Martin']})
        df_modified = remove_titles(df, 'Full Name')
        expected_names = ['J.R.R. Tolkien', 'G.R.R. Martin']
        self.assertEqual(list(df_modified['Full Name']), expected_names)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)